<a href="https://colab.research.google.com/github/CosmicQuant/activeloop-course/blob/main/Using_the_Open_Source_GPT4All_Model_Locally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'

# send a GET request to the URL to download the file.
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response
# to it in chunks.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

514266it [14:37, 585.75it/s]


In [4]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00


In [5]:
! git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 15464, done.
remote: Counting objects: 100% (5452/5452), done.
remote: Compressing objects: 100% (318/318), done.
remote: Total 15464 (delta 5309), reused 5161 (delta 5134), pack-reused 10012
Receiving objects: 100% (15464/15464), 18.15 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (10804/10804), done.


In [6]:
! cd llama.cpp && git checkout 2b26469

Note: switching to '2b26469'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2b26469 convert.py: Support models which are stored in a single pytorch_model.bin (#1469)


In [7]:
! python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

Loading model file models/gpt4all-lora-quantized-ggml.bin
Writing vocab...
[  1/291] Writing tensor tok_embeddings.weight                  | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  2/291] Writing tensor norm.weight                            | size   4096           | type UnquantizedDataType(name='F32')
[  3/291] Writing tensor output.weight                          | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  4/291] Writing tensor layers.0.attention.wq.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  5/291] Writing tensor layers.0.attention.wk.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  6/291] Writing tensor layers.0.attention.wv.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addend

In [8]:
! pip install -q langchain==0.0.152 pyllamacpp==1.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.9/666.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 28.7 MB/s eta 0:00:00


In [9]:
from langchain.llms import GPT4All
from langchain.llms.llamacpp import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [10]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [11]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model="./models/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True)
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What happens when it rains somewhere?"
llm_chain.run(question)

In [13]:
template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain.run(question)